PART 2
FROM PART 1, THE DATA HAS BEEN SAVED TO CSV. TO USE HERE IN PART 2

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np
from sklearn.cluster import KMeans

import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import requests

import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

import folium # Libarary that contains a detailed world map and the ability to zoom in down to the city level
from geopy.geocoders import Nominatim # reverse geocoder API to get coordinates for addresses/cites, etc.

print('All libraries are loaded.')


All libraries are loaded.


In [4]:
DF = pd.read_csv(r'PostalCoordinates.csv')
DF

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park / Ontario Provincial Government,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
7,M3B,North York,Don MillsNorth,43.745906,-79.352188
8,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [5]:
print('The full dataset has {} post code areas.'.format(DF.shape[0]))

The full dataset has 103 post code areas.


BOROUGHS THAT CONTAINS TORONTO

In [6]:
dfToronto = DF[DF['Borough'].str.contains('Toronto')].reset_index(drop=True)
dfToronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Richmond , Adelaide , King",43.650571,-79.384568
8,M6H,West Toronto,"Dufferin , Dovercourt Village",43.669005,-79.442259
9,M4J,East YorkEast Toronto,The Danforth East,43.685347,-79.338106


In [7]:
print('The main Toronto dataset has {} post code areas.'.format(dfToronto.shape[0]))

The main Toronto dataset has 39 post code areas.


FOLIUM LIB IS USED TO MAP TORONTO

In [8]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent = 'Toronto_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical cooridnates of Toronto, Ontario are Latitude: {}, Longitude: {}.'.format(latitude,longitude))

The geographical cooridnates of Toronto, Ontario are Latitude: 43.653963, Longitude: -79.387207.


In [9]:
map_toronto = folium.Map(location = [latitude, longitude], zoom_start=12, control_scale=True)

for lat, lng, postcode, borough, neighborhood in zip(dfToronto['Latitude'], dfToronto['Longitude'], dfToronto['PostalCode'], dfToronto['Borough'], dfToronto['Neighborhood']):
    label = '{}, {}, {}'.format(neighborhood,borough, postcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_toronto)

map_toronto # bring up the map